In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 41.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.8/680.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.4/635.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 39.9 MB/s eta 0:00:0000:01
  Created wheel for htbuilder: filename=htbuilder-0.9.0-py3-none-any.whl size=12840 sha256=ba7e6605e64882e73fe874aae7985c02991354b047ed664594be918416ac0d78
  Stored in directory: /Users/gorgegao/Library/Caches/pip/wheels/e1/3b/d9/19326152

In [2]:
!pip install -U langchain-huggingface

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
# ✅ Check if Ollama is running
import requests

try:
    response = requests.get("http://localhost:11434")
    if response.status_code == 404:
        print("✅ Ollama is running.")
    else:
        print("⚠️ Unexpected response. Ollama might still be running, but check logs.")
except requests.exceptions.ConnectionError:
    raise RuntimeError("❌ Ollama server is not running. Please open a terminal and run: `ollama run mistral`")

⚠️ Unexpected response. Ollama might still be running, but check logs.


/Users/gorgegao/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 📓 Ask My PDFs – Demo Notebook
This notebook shows how to:
- Embed multiple PDFs
- Run retrieval-augmented generation queries
- Use local LLMs via Ollama

In [7]:
from rag_utils import load_and_split_pdfs, embed_documents, query_ollama
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Step 1: Load and embed documents
docs = load_and_split_pdfs("pdfs")
# vectordb = embed_documents(docs, persist_dir="chroma_db")
vectordb = embed_documents(docs)

# Step 2: Ask a question
retriever = vectordb.as_retriever()
query = "What is the main topic of the paper?"
relevant_docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in relevant_docs[:3]])

# Step 3: Generate answer using Ollama
response = query_ollama(context, query)
print("Answer:", response)

Answer: There is no paper mentioned in the provided context. The text appears to be a summary of educational background and professional experience.


In [8]:
# Ask a question
query = "Can you summarize their professional background?"
relevant_docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in relevant_docs[:3]])

# Generate answer using Ollama
response = query_ollama(context, query)
print("Answer:", response)

Answer: Based on the provided context, here is a summary of Jie Gao's professional background:

* Currently working as a Senior Data Scientist at Walmart in Hoboken, New Jersey (since September 2021)
* Has over 5 years of experience in customer and marketing analysis, data integrity, modeling, and visualization
* Awarded "Driven of Result" under in 2023 Winter, rated exemplary during the 1st year in Walmart


In [9]:
# Ask a question
query = "What is this candidate's education level"
relevant_docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in relevant_docs[:3]])

# Generate answer using Ollama
response = query_ollama(context, query)
print("Answer:", response)

Answer: Master's degree. The candidate has a Master in Econometrics and Quantitative Economics from New York University, completed from September 2018 to May 2020.


In [10]:
# Ask a question
query = "Does this candidate has MMM experience"
relevant_docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in relevant_docs[:3]])

# Generate answer using Ollama
response = query_ollama(context, query)
print("Answer:", response)

Answer: Yes, according to the context, Jie Gao has MMM (Marketing Mix Modeling) experience. The summary mentions:

* Developed an in-house Bayesian Hierarchy MMM solution to measure the incremental media contribution on driving High Value Action customers.
* Utilized MMM to optimize marketing budgets, achieving an average 3% increase in marketing ROI.

This indicates that Jie Gao has hands-on experience with MMM and its applications in optimizing marketing budgets.
